In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scipy_solver.linprog import linprog
from tqdm import tqdm

import gzip
import pickle
import torch
from scipy.linalg import LinAlgWarning
from scipy.optimize._optimize import OptimizeWarning
import warnings
import numpy as np
from functools import partial

from generate_instances import generate_setcover, Graph, generate_indset, generate_cauctions, generate_capacited_facility_location

In [14]:
rng = np.random.RandomState(1)

In [15]:
bounds = (0., 1.)

In [16]:
root = '/mnt/d/ineq_fac_instances_7'

### Setcover

In [ ]:
density=0.05
nrows_l = 900
nrows_u = 1100
ncols_l = 500
ncols_u = 700

surrogate_gen = partial(generate_setcover, nrows_l=nrows_l, nrows_u=nrows_u, ncols_l=ncols_l, ncols_u=ncols_u, density=density, rng=rng)

### Indset

In [ ]:
def surrogate_gen():
    nnodes = rng.randint(600, 800)
    graph = Graph.barabasi_albert(number_of_nodes=nnodes, affinity=2, random=rng)
    A, b, c = generate_indset(graph=graph, nnodes=nnodes)
    return A, b, c

### Cauctions

In [ ]:
n_items=600
n_bids=500

surrogate_gen = partial(generate_cauctions, n_items=n_items, n_bids=n_bids, rng=rng, min_value=0.001, max_value=0.1)

### Facilities

In [17]:
def surrogate_gen():
    n_customers = np.random.randint(30, 35)
    n_facilities = np.random.randint(20, 30)
    ratio = 5
    A, b, c = generate_capacited_facility_location(n_customers=n_customers, n_facilities=n_facilities, ratio=ratio, rng=rng)
    return A, b, c

## create

In [ ]:
warnings.filterwarnings("error")

ips = []
pkg_idx = 0
success_cnt = 0
fail_cnt = 0

max_iter = 15000
num = 1000

for i in tqdm(range(max_iter)):
    A, b, c = surrogate_gen()
    
    try:
        A_eq = None
        b_eq = None
        A_ub = A
        b_ub = b
        res = linprog(c, 
                A_ub=A_ub,
                b_ub=b_ub,
                A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='interior-point')
    except (LinAlgWarning, OptimizeWarning, AssertionError):
        fail_cnt += 1
        continue
    else:
        if res.success and not np.isnan(res.fun):
            ips.append((torch.from_numpy(A).to(torch.float), torch.from_numpy(b).to(torch.float), torch.from_numpy(c).to(torch.float)))
            success_cnt += 1

    if len(ips) >= 1000 or success_cnt == num:
        with gzip.open(f'{root}/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            pkg_idx += 1
        ips = []

    if success_cnt >= num:
        break

warnings.resetwarnings()

## create and graphize

In [7]:
import os
import os.path as osp
from typing import Callable, List, Optional

from torch_geometric.data import Data, Batch, HeteroData, InMemoryDataset
from torch_sparse import SparseTensor

In [8]:
from torch_geometric.transforms import Compose
from data.data_preprocess import HeteroAddLaplacianEigenvectorPE, SubSample

In [9]:
pre_transform=Compose([HeteroAddLaplacianEigenvectorPE(k=5),
                                                     SubSample(8)])

In [ ]:
warnings.filterwarnings("error")

ips = []
data_list = []
pkg_idx = 0
pkg_size = 100
success_cnt = 0
fail_cnt = 0

max_iter = 120
num = 100

for i in tqdm(range(max_iter)):
    A, b, c = surrogate_gen()
    c = c / (np.abs(c).max() + 1.e-10)  # does not change the result
    
    try:
        A_eq = None
        b_eq = None
        A_ub = A
        b_ub = b
        
        sol = linprog(c,
                      A_ub=A_ub,
                      b_ub=b_ub,
                      A_eq=A_eq, b_eq=b_eq, bounds=bounds,
                      method='interior-point', callback=lambda res: res.x)
        
    except (LinAlgWarning, OptimizeWarning, AssertionError):
        fail_cnt += 1
        continue
    else:
        if sol.success and not np.isnan(sol.fun):
            ips.append((torch.from_numpy(A).to(torch.float), torch.from_numpy(b).to(torch.float), torch.from_numpy(c).to(torch.float)))
            success_cnt += 1
            
            (A, b, c) = ips[-1]
            
            sp_a = SparseTensor.from_dense(A, has_value=True)
            
            row = sp_a.storage._row
            col = sp_a.storage._col
            val = sp_a.storage._value

            tilde_mask = torch.ones(row.shape, dtype=torch.bool)
            
            x = np.stack(sol.intermediate, axis=1)
            gt_primals = torch.from_numpy(x).to(torch.float)
            
            
            data = HeteroData(
                cons={'x': torch.cat([A.mean(1, keepdims=True),
                                      A.std(1, keepdims=True)], dim=1)},
                vals={'x': torch.cat([A.mean(0, keepdims=True),
                                      A.std(0, keepdims=True)], dim=0).T},
                obj={'x': torch.cat([c.mean(0, keepdims=True),
                                     c.std(0, keepdims=True)], dim=0)[None]},

                cons__to__vals={'edge_index': torch.vstack(torch.where(A)),
                                'edge_attr': A[torch.where(A)][:, None]},
                vals__to__cons={'edge_index': torch.vstack(torch.where(A.T)),
                                'edge_attr': A.T[torch.where(A.T)][:, None]},
                vals__to__obj={'edge_index': torch.vstack([torch.arange(A.shape[1]),
                                                           torch.zeros(A.shape[1], dtype=torch.long)]),
                               'edge_attr': c[:, None]},
                obj__to__vals={'edge_index': torch.vstack([torch.zeros(A.shape[1], dtype=torch.long),
                                                           torch.arange(A.shape[1])]),
                               'edge_attr': c[:, None]},
                cons__to__obj={'edge_index': torch.vstack([torch.arange(A.shape[0]),
                                                           torch.zeros(A.shape[0], dtype=torch.long)]),
                               'edge_attr': b[:, None]},
                obj__to__cons={'edge_index': torch.vstack([torch.zeros(A.shape[0], dtype=torch.long),
                                                           torch.arange(A.shape[0])]),
                               'edge_attr': b[:, None]},
                gt_primals=gt_primals,
                # gt_duals=gt_duals,
                # gt_slacks=gt_slacks,
                obj_value=torch.tensor(sol['fun'].astype(np.float32)),
                obj_const=c,

                A_row=row,
                A_col=col,
                A_val=val,
                A_num_row=A.shape[0],
                A_num_col=A.shape[1],
                A_nnz=len(val),
                A_tilde_mask=tilde_mask,
                rhs=b)
            
            data = pre_transform(data)
            data_list.append(data)

    if len(ips) >= pkg_size or success_cnt == num:
        with gzip.open(f'{root}/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            
        ips = []
        
        torch.save(Batch.from_data_list(data_list), osp.join(f'{root}/processed_1restarts_5lap_8steps_upper_1.0', f'batch{pkg_idx}.pt'))
        data_list = []
        
        pkg_idx += 1

    if success_cnt >= num:
        break

warnings.resetwarnings()